In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [26]:
from citylearn.citylearn import CityLearnEnv
from agents.orderenforcingwrapper import OrderEnforcingAgent
import numpy as np
import pandas as pd
import time
from input_function import observation_no_forecast_added_hour_range as input_func
from input_function import active_observations

In [34]:
class Constants:
    episodes = 1
    schema_path = "./data/citylearn_challenge_2022_phase_1/schema.json"

def action_space_to_dict(aspace):
    """Only for box space"""
    return {
        "high": aspace.high,
        "low": aspace.low,
        "shape": aspace.shape,
        "dtype": str(aspace.dtype),
    }

def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    building_info = env.get_building_information()
    building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {
        "action_space": action_space_dicts,
        "observation_space": observation_space_dicts,
        "building_info": building_info,
        "observation": observations,
    }
    return obs_dict


class Env:
    def __init__(self) -> None:
        self.time_step = 0
        self.prev_steps = dict()
        

    def populate_prev_steps(self, observations):
        done_keys = list()
        new_obs, obs_info = input_func(observations)
        num_buildings = len(observations)

        # Add history of observations
        for i, obs in enumerate(observations[0]):
            if not 20 <= i < 23:
                key = active_observations[i]
                done_keys.append(key)
                if key not in self.prev_steps.keys():
                    self.prev_steps[key] = [obs]
                else:
                    self.prev_steps[key].append(obs)

        for i in range(num_buildings):
            for j in range(20, 24):
                key = f"{active_observations[j]}_{i}"
                done_keys.append(key)
                obs = observations[i][j]
                if key not in self.prev_steps.keys():
                    self.prev_steps[key] = [obs]
                else:
                    self.prev_steps[key].append(obs)

        for i, obs in enumerate(new_obs):
            key = obs_info[i][0]
            if key not in done_keys:
                if key not in self.prev_steps.keys():
                    self.prev_steps[key] = [obs]
                else:
                    self.prev_steps[key].append(obs)

        key = "action"

        if key not in self.prev_steps.keys():
            self.prev_steps[key] = [action]
        else:
            self.prev_steps[key].append(action)

    def update_time_step(self):
        self.time_step += 1

    def iterate_step(self, observations):
        self.update_time_step()
        self.time_step = self.time_step % 8760
        num_buildings = len(observations)
        batt_socs = [observations[i][22] for i in range(num_buildings)]
        all_actions = []
        _, obs_info = input_func(self, observations)


In [36]:
env = CityLearnEnv(schema=Constants.schema_path)
agent = OrderEnforcingAgent()
env_agent = Env()
obs_dict = env_reset(env)
agent_time_elapsed = 0
step_start = time.perf_counter()
actions = agent.register_reset(obs_dict)
agent_time_elapsed += time.perf_counter() - step_start

episodes_completed = 0
num_steps = 0
interrupted = False
episode_metrics = []

In [37]:
input_func = observation_no_forecast_added_hour_range
obs_space_zero = [[0] * 28]
_, feature_names = input_func(None, obs_space_zero)

In [39]:
observations, _, done, _ = env.step(actions)
step_start = time.perf_counter()
actions = env_agent.iterate_step(observations)
agent_time_elapsed += time.perf_counter() - step_start           
num_steps += 1
print(observations)
if num_steps % 1000 == 0:
    # log_usefull(env, filename)
    print(f"Num Steps: {num_steps}, Num episodes: {episodes_completed}")
    """
    metrics_t = env.evaluate()
    print(metrics_t)
    metrics = {"price_cost": metrics_t[0], "emmision_cost": metrics_t[1]}

    print(
        "Average Price Cost:", np.mean([e["price_cost"] for e in episode_metrics])
    )
    print(
        "Average Emmision Cost:",
        np.mean([e["emmision_cost"] for e in episode_metrics]),
    )
    print("Average Grid Cost:", np.mean([e["grid_cost"] for e in episode_metrics]))
    """


AttributeError: 'Env' object has no attribute 'time_step'

In [25]:
feature_names

[['month', [0.0, 13.0]],
 ['day_type', [0.0, 8.0]],
 ['hour', [0.0, 25.0]],
 ['carbon_intensity', [0.07, 0.2817962]],
 ['electricity_pricing', [0.21, 0.54]],
 ['average_non_shiftable_load', [-10.0, 10.0]],
 ['average_solar_generation', [-1.0, 7.0]],
 ['average_electrical_storage_soc', [-1.0, 2.0]],
 ['average_net_electricity_consumption', [-10.0, 10.0]],
 ['non_shiftable_load_0', [-10.0, 10.0]],
 ['solar_generation_0', [-1.0, 7.0]],
 ['electrical_storage_soc_0', [-1.0, 2.0]],
 ['net_electricity_consumption_0', [-10.0, 10.0]],
 ['Hour range', [0.0, 24.0]]]